In [4]:
import socket
import json
import threading
from uuid import uuid4
import datetime


class Conversation:
    def __init__(self, npc, location, topic, iterrupt_speech):
        self.npc = npc
        self.location = location
        self.topic = topic
        self.iterrupt_speech = iterrupt_speech
        self.id = str(uuid4())

    def add_memory(self, conversation_id, index):
        # Add memory logic here
        print("add_memory:",conversation_id, index, datetime.datetime.now())
        pass

    def generate_script(self):
        # Generate script logic here
        script = {
            "name": "conversation",
            "id": self.id,
            "length": 24,
            "lines": [
                # Add conversation lines here
            ]
        }
        return script


class NPCEngine:
    def __init__(self, listen_port=8199, target_url="127.0.0.1", target_port=8084):
        self.port = listen_port
        self.target_url = target_url
        self.target_port = target_port
        self.conversation_dict = {}
        self.sock = socket.socket(socket.AF_INET6, socket.SOCK_DGRAM)  # 使用IPv6地址
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # 添加这一行
        self.sock.bind(('::', self.port))  # 修改为IPv6地址绑定方式
        self.listen_thread = threading.Thread(target=self.listen)
        self.listen_thread.start()

    def listen(self):
        print(f"listening on [::]:{self.port}")
        while True:
            data, addr = self.sock.recvfrom(1024)
            try:
                json_data = json.loads(data.decode())
                if "func" in json_data:
                    func_name = json_data["func"]
                    if hasattr(self, func_name):
                        func = getattr(self, func_name)
                        func(json_data)
            except json.JSONDecodeError:
                pass

    def conversation(self, json_data):
        # 获取参数
        npc = json_data["npc"]
        location = json_data["location"]
        topic = json_data["topic"]
        iterrupt_speech = json_data["iterrupt_speech"]
        # 实例化Conversation
        convo = Conversation(npc, location, topic, iterrupt_speech)
        self.conversation_dict[convo.id] = convo
        script = convo.generate_script()
        self.send_script(script)

    def confirm_conversation_line(self, json_data):
        conversation_id = json_data["conversation_id"]
        index = json_data["index"]
        if conversation_id in self.conversation_dict:
            convo = self.conversation_dict[conversation_id]
            convo.add_memory(conversation_id, index)

    def send_script(self, script):
        self.sock.sendto(json.dumps(script).encode(), (self.target_url, self.target_port))


if __name__ == "__main__":
    npc_engine = NPCEngine()